# Assignment Data Eng.

# Task One


## Extracting Links / Sending Request

In [1]:
import requests
from bs4 import BeautifulSoup
import json

headers = {'Accept': '/', 'Connection': 'keep-alive', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/537.36 (KHTML,like Gecko) Chrome/70.0.3538.110 Safari/537.36', 'Accept-Language':'en-US;q=0.5,en;q=0.3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1'}

response = requests.get('https://www.imdb.com/chart/top/', headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

data = json.loads(soup.find('script', {'type':"application/ld+json"}).text)
print(len(data['itemListElement']))
for item in data['itemListElement'][:250]:
    print(item['item']['url'])

250
https://www.imdb.com/title/tt0111161/
https://www.imdb.com/title/tt0068646/
https://www.imdb.com/title/tt0468569/
https://www.imdb.com/title/tt0071562/
https://www.imdb.com/title/tt0050083/
https://www.imdb.com/title/tt0167260/
https://www.imdb.com/title/tt0108052/
https://www.imdb.com/title/tt0110912/
https://www.imdb.com/title/tt0120737/
https://www.imdb.com/title/tt0060196/
https://www.imdb.com/title/tt0109830/
https://www.imdb.com/title/tt0167261/
https://www.imdb.com/title/tt0137523/
https://www.imdb.com/title/tt1375666/
https://www.imdb.com/title/tt0080684/
https://www.imdb.com/title/tt0133093/
https://www.imdb.com/title/tt0099685/
https://www.imdb.com/title/tt0073486/
https://www.imdb.com/title/tt0816692/
https://www.imdb.com/title/tt0114369/
https://www.imdb.com/title/tt0038650/
https://www.imdb.com/title/tt0047478/
https://www.imdb.com/title/tt0102926/
https://www.imdb.com/title/tt0120815/
https://www.imdb.com/title/tt0317248/
https://www.imdb.com/title/tt0120689/
https://

## Extracting full cast and crew

In [2]:

import pandas as pd
import time


headers = {
    'Accept': '/',
    'Connection': 'keep-alive',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/537.36 (KHTML,like Gecko) Chrome/70.0.3538.110 Safari/537.36',
}


def fetch_cast_and_crew(movie_url, movie_id):
    cast_and_crew = []
    try:
        
        response = requests.get(movie_url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        
        full_cast_link = None
        for link in soup.find_all("a", href=True):
            if "fullcredits" in link['href']:
                full_cast_link = "https://www.imdb.com" + link['href']
                break

        if full_cast_link:
            
            cast_response = requests.get(full_cast_link, headers=headers)
            cast_soup = BeautifulSoup(cast_response.content, 'html.parser')

            
            cast_table = cast_soup.find("table", class_="cast_list")
            if cast_table:
                for row in cast_table.find_all("tr")[1:]:  
                    cols = row.find_all("td")
                    if len(cols) >= 2:
                        name = cols[1].text.strip()
                        character = cols[3].text.strip() if len(cols) > 3 else "N/A"
                        cast_and_crew.append({
                            "Movie_ID": movie_id,
                            "Name": name,
                            "Role Type": "Actor",
                            "Character/Position": character
                        })

           
            produced_by_section = cast_soup.find("h4", string="Produced by")
            if produced_by_section:
                for producer in produced_by_section.find_next("ul").find_all("li"):
                    name = producer.text.strip()
                    cast_and_crew.append({
                        "Movie_ID": movie_id,
                        "Name": name,
                        "Role Type": "Producer",
                        "Character/Position": "N/A"
                    })
        else:
            print(f"No Full Cast & Crew link found for {movie_url}")
    
    except Exception as e:
        print(f"Error scraping cast & crew for {movie_url}: {e}")

    return cast_and_crew



movie_urls = [
    "https://www.imdb.com/title/tt0111161/",  
    "https://www.imdb.com/title/tt0068646/", 
    
]


movie_urls = [
    f"https://www.imdb.com/title/tt{1000000 + i}/" for i in range(1, 251)
]


full_cast_and_crew_list = []

 
for movie_url in movie_urls:
    movie_id = movie_url.split("/")[4] 
    print(f"Fetching cast and crew for movie ID {movie_id} from {movie_url}")
    
    
    cast_and_crew = fetch_cast_and_crew(movie_url, movie_id)
    
    
    full_cast_and_crew_list.extend(cast_and_crew)
    
    
    time.sleep(1)


full_cast_and_crew_df = pd.DataFrame(full_cast_and_crew_list)


print(full_cast_and_crew_df.head())


full_cast_and_crew_df.to_csv("full_cast_and_crew.csv", index=False)

Fetching cast and crew for movie ID tt1000001 from https://www.imdb.com/title/tt1000001/
Fetching cast and crew for movie ID tt1000002 from https://www.imdb.com/title/tt1000002/
No Full Cast & Crew link found for https://www.imdb.com/title/tt1000002/
Fetching cast and crew for movie ID tt1000003 from https://www.imdb.com/title/tt1000003/
Fetching cast and crew for movie ID tt1000004 from https://www.imdb.com/title/tt1000004/
Fetching cast and crew for movie ID tt1000005 from https://www.imdb.com/title/tt1000005/
Fetching cast and crew for movie ID tt1000006 from https://www.imdb.com/title/tt1000006/
Fetching cast and crew for movie ID tt1000007 from https://www.imdb.com/title/tt1000007/
Fetching cast and crew for movie ID tt1000008 from https://www.imdb.com/title/tt1000008/
Fetching cast and crew for movie ID tt1000009 from https://www.imdb.com/title/tt1000009/
Fetching cast and crew for movie ID tt1000010 from https://www.imdb.com/title/tt1000010/
Fetching cast and crew for movie ID t

Fetching cast and crew for movie ID tt1000083 from https://www.imdb.com/title/tt1000083/
Fetching cast and crew for movie ID tt1000084 from https://www.imdb.com/title/tt1000084/
Fetching cast and crew for movie ID tt1000085 from https://www.imdb.com/title/tt1000085/
Fetching cast and crew for movie ID tt1000086 from https://www.imdb.com/title/tt1000086/
Fetching cast and crew for movie ID tt1000087 from https://www.imdb.com/title/tt1000087/
Fetching cast and crew for movie ID tt1000088 from https://www.imdb.com/title/tt1000088/
Fetching cast and crew for movie ID tt1000089 from https://www.imdb.com/title/tt1000089/
Fetching cast and crew for movie ID tt1000090 from https://www.imdb.com/title/tt1000090/
Fetching cast and crew for movie ID tt1000091 from https://www.imdb.com/title/tt1000091/
Fetching cast and crew for movie ID tt1000092 from https://www.imdb.com/title/tt1000092/
Fetching cast and crew for movie ID tt1000093 from https://www.imdb.com/title/tt1000093/
Fetching cast and cre

Fetching cast and crew for movie ID tt1000173 from https://www.imdb.com/title/tt1000173/
Fetching cast and crew for movie ID tt1000174 from https://www.imdb.com/title/tt1000174/
Fetching cast and crew for movie ID tt1000175 from https://www.imdb.com/title/tt1000175/
Fetching cast and crew for movie ID tt1000176 from https://www.imdb.com/title/tt1000176/
Fetching cast and crew for movie ID tt1000177 from https://www.imdb.com/title/tt1000177/
Fetching cast and crew for movie ID tt1000178 from https://www.imdb.com/title/tt1000178/
Fetching cast and crew for movie ID tt1000179 from https://www.imdb.com/title/tt1000179/
Fetching cast and crew for movie ID tt1000180 from https://www.imdb.com/title/tt1000180/
Fetching cast and crew for movie ID tt1000181 from https://www.imdb.com/title/tt1000181/
Fetching cast and crew for movie ID tt1000182 from https://www.imdb.com/title/tt1000182/
Fetching cast and crew for movie ID tt1000183 from https://www.imdb.com/title/tt1000183/
Fetching cast and cre

In [16]:
import requests
from bs4 import BeautifulSoup
import json
import time
import pandas as pd

headers = {
    'Accept': '/',
    'Connection': 'keep-alive',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/537.36 (KHTML,like Gecko) Chrome/70.0.3538.110 Safari/537.36',
    'Accept-Language': 'en-US;q=0.5,en;q=0.3',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1'
}

response = requests.get('https://www.imdb.com/chart/top/', headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')


data = json.loads(soup.find('script', {'type': "application/ld+json"}).text)
movie_urls = [item['item']['url'] for item in data['itemListElement'][:250]]

print(f"Found {len(movie_urls)} movie URLs.")


def fetch_cast_and_crew(movie_url, movie_id):
    cast_and_crew = []
    try:
        response = requests.get(movie_url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        full_cast_link = None
        for link in soup.find_all("a", href=True):
            if "fullcredits" in link['href']:
                full_cast_link = "https://www.imdb.com" + link['href']
                break

        if full_cast_link:
            cast_response = requests.get(full_cast_link, headers=headers)
            cast_soup = BeautifulSoup(cast_response.content, 'html.parser')

            cast_table = cast_soup.find("table", class_="cast_list")
            if cast_table:
                for row in cast_table.find_all("tr")[1:]:
                    cols = row.find_all("td")
                    if len(cols) >= 2:
                        name = cols[1].text.strip()
                        character = cols[3].text.strip() if len(cols) > 3 else "N/A"
                        cast_and_crew.append({
                            "Movie_ID": movie_id,
                            "Name": name,
                            "Role Type": "Actor",
                            "Character/Position": character
                        })

            
            produced_by_section = cast_soup.find("h4", string="Produced by")
            if produced_by_section:
                for producer in produced_by_section.find_next("ul").find_all("li"):
                    name = producer.text.strip()
                    cast_and_crew.append({
                        "Movie_ID": movie_id,
                        "Name": name,
                        "Role Type": "Producer",
                        "Character/Position": "N/A"
                    })
        else:
            print(f"No Full Cast & Crew link found for {movie_url}")

    except Exception as e:
        print(f"Error scraping cast & crew for {movie_url}: {e}")

    return cast_and_crew


full_cast_and_crew_list = []


for movie_url in movie_urls:
    movie_id = movie_url.split("/")[4] 
    print(f"Fetching cast and crew for movie ID {movie_id} from {movie_url}")

    cast_and_crew = fetch_cast_and_crew(movie_url, movie_id)
    full_cast_and_crew_list.extend(cast_and_crew)

    time.sleep(1)  


full_cast_and_crew_df = pd.DataFrame(full_cast_and_crew_list)


print(full_cast_and_crew_df.head())


full_cast_and_crew_df.to_csv("full_cast_and_crew.csv", index=False)

print("Full cast and crew data saved to 'full_cast_and_crew.csv'")


Found 250 movie URLs.
Fetching cast and crew for movie ID tt0111161 from https://www.imdb.com/title/tt0111161/
Fetching cast and crew for movie ID tt0068646 from https://www.imdb.com/title/tt0068646/
Fetching cast and crew for movie ID tt0468569 from https://www.imdb.com/title/tt0468569/
Fetching cast and crew for movie ID tt0071562 from https://www.imdb.com/title/tt0071562/
Fetching cast and crew for movie ID tt0050083 from https://www.imdb.com/title/tt0050083/
Fetching cast and crew for movie ID tt0167260 from https://www.imdb.com/title/tt0167260/
Fetching cast and crew for movie ID tt0108052 from https://www.imdb.com/title/tt0108052/
Fetching cast and crew for movie ID tt0110912 from https://www.imdb.com/title/tt0110912/
Fetching cast and crew for movie ID tt0120737 from https://www.imdb.com/title/tt0120737/
Fetching cast and crew for movie ID tt0060196 from https://www.imdb.com/title/tt0060196/
Fetching cast and crew for movie ID tt0109830 from https://www.imdb.com/title/tt0109830/

Fetching cast and crew for movie ID tt0086190 from https://www.imdb.com/title/tt0086190/
Fetching cast and crew for movie ID tt2106476 from https://www.imdb.com/title/tt2106476/
Fetching cast and crew for movie ID tt0338013 from https://www.imdb.com/title/tt0338013/
Fetching cast and crew for movie ID tt0044741 from https://www.imdb.com/title/tt0044741/
Fetching cast and crew for movie ID tt0062622 from https://www.imdb.com/title/tt0062622/
Fetching cast and crew for movie ID tt0105236 from https://www.imdb.com/title/tt0105236/
Fetching cast and crew for movie ID tt0056172 from https://www.imdb.com/title/tt0056172/
Fetching cast and crew for movie ID tt0053604 from https://www.imdb.com/title/tt0053604/
Fetching cast and crew for movie ID tt1255953 from https://www.imdb.com/title/tt1255953/
Fetching cast and crew for movie ID tt0086250 from https://www.imdb.com/title/tt0086250/
Fetching cast and crew for movie ID tt0053125 from https://www.imdb.com/title/tt0053125/
Fetching cast and cre

Fetching cast and crew for movie ID tt0118849 from https://www.imdb.com/title/tt0118849/
Fetching cast and crew for movie ID tt0072684 from https://www.imdb.com/title/tt0072684/
Fetching cast and crew for movie ID tt2024544 from https://www.imdb.com/title/tt2024544/
Fetching cast and crew for movie ID tt0083658 from https://www.imdb.com/title/tt0083658/
Fetching cast and crew for movie ID tt0112471 from https://www.imdb.com/title/tt0112471/
Fetching cast and crew for movie ID tt2278388 from https://www.imdb.com/title/tt2278388/
Fetching cast and crew for movie ID tt2119532 from https://www.imdb.com/title/tt2119532/
Fetching cast and crew for movie ID tt0097165 from https://www.imdb.com/title/tt0097165/
Fetching cast and crew for movie ID tt2267998 from https://www.imdb.com/title/tt2267998/
Fetching cast and crew for movie ID tt0353969 from https://www.imdb.com/title/tt0353969/
Fetching cast and crew for movie ID tt26548265 from https://www.imdb.com/title/tt26548265/
Fetching cast and c

## Extracting Top 250 Imbd Movies

In [9]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time


headers = {
    'Accept': '/',
    'Connection': 'keep-alive',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/537.36 (KHTML,like Gecko) Chrome/70.0.3538.110 Safari/537.36',
    'Accept-Language': 'en-US;q=0.5,en;q=0.3',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1'
}

response = requests.get('https://www.imdb.com/chart/top/', headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')


data = json.loads(soup.find('script', {'type': "application/ld+json"}).text)


movie_urls = [item['item']['url'] for item in data['itemListElement'][:250]]

movies = []


for url in movie_urls:  # 
    movie_response = requests.get(url, headers=headers)
    movie_soup = BeautifulSoup(movie_response.content, 'html.parser')

    try:
       
        movie_id = url.split('/')[4]  

        
        title = movie_soup.find("h1").text.strip()

        
        release_year = "N/A"
        release_date = "N/A"
        json_ld = movie_soup.find('script', type='application/ld+json')
        if json_ld:
            movie_data = json.loads(json_ld.text)
            release_year = movie_data.get('datePublished', "N/A")[:4] if 'datePublished' in movie_data else "N/A"
            release_date = movie_data.get('datePublished', "N/A")

        
        rating_section = movie_soup.find("div", {"data-testid": "hero-rating-bar__aggregate-rating__score"})
        rating = rating_section.text.strip() if rating_section else "N/A"

        
        director = ", ".join([d['name'] for d in movie_data.get('director', [])]) if 'director' in movie_data else "N/A"
        genres = ", ".join(movie_data.get('genre', [])) if 'genre' in movie_data else "N/A"

        
        details_section = movie_soup.find("section", {"data-testid": "Details"})
        country, language, filming_locations = "N/A", "N/A", "N/A"

        if details_section:
           
            country_section = details_section.find("li", {"data-testid": "title-details-origin"})
            if country_section:
                country_links = country_section.find_all("a")
                countries = [link.text.strip() for link in country_links if link.text.strip()]
                country = ", ".join(countries) if countries else "N/A"

           
            language_section = details_section.find("li", {"data-testid": "title-details-languages"})
            if language_section:
                language_links = language_section.find_all("a")
                languages = [link.text.strip() for link in language_links if link.text.strip()]
                language = ", ".join(languages) if languages else "N/A"

            
            filming_locations_tag = details_section.find("li", {"data-testid": "title-details-filminglocations"})
            if filming_locations_tag:
                filming_locations = filming_locations_tag.text.strip()

       
        box_office_section = movie_soup.find("section", {"data-testid": "BoxOffice"})
        budget = "N/A"
        gross_revenue = "N/A"
        
        if box_office_section:
            budget_tag = box_office_section.find(string="Budget")
            budget = budget_tag.find_next("div").text.strip() if budget_tag else "N/A"

            gross_tag = box_office_section.find(string="Gross worldwide")
            gross_revenue = gross_tag.find_next("div").text.strip() if gross_tag else "N/A"

        
        official_sites_tag = movie_soup.find("li", {"data-testid": "details-officialsites"})
        official_sites = official_sites_tag.text.strip() if official_sites_tag else "N/A"

       
        production_companies = "N/A"
        production_section = details_section.find("li", {"data-testid": "title-details-companies"}) if details_section else None
        if production_section:
            production_links = production_section.find_all("a")
            companies = [link.text.strip() for link in production_links if link.text.strip()]
            production_companies = ", ".join(companies) if companies else "N/A"

       
        movies.append({
            "Movie_ID": movie_id,
            "Title": title,
            "Release Year": release_year,
            "Release Date": release_date,
            "Rating": rating,
            "Director(s)": director,
            "Genre(s)": genres,
            "Country of Origin": country,
            "Language": language,
            "Filming Locations": filming_locations,
            "Budget": budget,
            "Gross Worldwide Revenue": gross_revenue,
            "Official Sites": official_sites,
            "Production Companies": production_companies
        })

    except Exception as e:
        print(f"Error scraping {url}: {e}")

 
    time.sleep(2)  


movies_df = pd.DataFrame(movies)
movies_df.to_csv("imdb_top_250_movies.csv", index=False)

print(movies_df.head())


    Movie_ID                     Title Release Year Release Date  Rating  \
0  tt0111161  The Shawshank Redemption         1994   1994-10-14  9.3/10   
1  tt0068646             The Godfather         1972   1972-03-24  9.2/10   
2  tt0468569           The Dark Knight         2008   2008-07-18  9.0/10   
3  tt0071562     The Godfather Part II         1974   1974-12-18  9.0/10   
4  tt0050083              12 Angry Men         1957   1957-04-10  9.0/10   

            Director(s)              Genre(s)              Country of Origin  \
0        Frank Darabont                 Drama                  United States   
1  Francis Ford Coppola          Crime, Drama                  United States   
2     Christopher Nolan  Action, Crime, Drama  United States, United Kingdom   
3  Francis Ford Coppola          Crime, Drama                  United States   
4          Sidney Lumet          Crime, Drama                  United States   

                                     Language  \
0            

# Task 2

## Clean The Scraped Data

### Clean The Imbd Table

In [11]:
import pandas as pd
import re


movies_df = pd.read_csv("imdb_top_250_movies.csv")


movies_df.fillna("N/A", inplace=True)


movies_df.drop_duplicates(inplace=True)


movies_df['Budget'] = movies_df['Budget'].apply(lambda x: re.sub(r'[^\d]', '', x) if x != "N/A" else x)
movies_df['Gross Worldwide Revenue'] = movies_df['Gross Worldwide Revenue'].apply(lambda x: re.sub(r'[^\d]', '', x) if x != "N/A" else x)


movies_df['Budget'] = pd.to_numeric(movies_df['Budget'], errors='coerce').fillna("N/A").astype(str)
movies_df['Gross Worldwide Revenue'] = pd.to_numeric(movies_df['Gross Worldwide Revenue'], errors='coerce').fillna("N/A").astype(str)


movies_df['Release Year'] = movies_df['Release Year'].apply(lambda x: str(x)[:4] if len(str(x)) > 4 else str(x))


for col in movies_df.select_dtypes(include=['object']):  
    movies_df[col] = movies_df[col].map(lambda x: x.strip() if isinstance(x, str) else x)


movies_df.to_csv("clean_imdb_top_250_movies.csv", index=False)


print(movies_df.head())

    Movie_ID                     Title Release Year Release Date  Rating  \
0  tt0111161  The Shawshank Redemption         1994   1994-10-14  9.3/10   
1  tt0068646             The Godfather         1972   1972-03-24  9.2/10   
2  tt0468569           The Dark Knight         2008   2008-07-18  9.0/10   
3  tt0071562     The Godfather Part II         1974   1974-12-18  9.0/10   
4  tt0050083              12 Angry Men         1957   1957-04-10  9.0/10   

            Director(s)              Genre(s)              Country of Origin  \
0        Frank Darabont                 Drama                  United States   
1  Francis Ford Coppola          Crime, Drama                  United States   
2     Christopher Nolan  Action, Crime, Drama  United States, United Kingdom   
3  Francis Ford Coppola          Crime, Drama                  United States   
4          Sidney Lumet          Crime, Drama                  United States   

                                     Language  \
0            

### Clean The Crew Table

In [12]:
import pandas as pd


input_file = "full_cast_and_crew.csv"
output_file = "clean_full_cast_and_crew.csv"

try:
   
    df = pd.read_csv(input_file)

   
    print("Initial Data Preview:")
    print(df.head())
    print("\nData Info:")
    print(df.info())

    
    df = df.drop_duplicates()

    
    df['Character/Position'] = df['Character/Position'].fillna('Unknown')

    
    df.columns = [col.strip().replace(" ", "_").lower() for col in df.columns]

  
    df.rename(columns={'movie_id': 'Movie_ID'}, inplace=True)

    
    df['role_type'] = df['role_type'].str.title()

   
    string_columns = df.select_dtypes(include=['object']).columns
    df[string_columns] = df[string_columns].apply(lambda x: x.str.strip())

    
    df.to_csv(output_file, index=False)
    print(f"\nCleaned data saved to {output_file}")

except FileNotFoundError:
    print(f"File {input_file} not found. Ensure the file is in the correct directory.")
except Exception as e:
    print(f"An error occurred during processing: {e}")


Initial Data Preview:
    Movie_ID            Name Role Type        Character/Position
0  tt0111161     Tim Robbins     Actor             Andy Dufresne
1  tt0111161  Morgan Freeman     Actor  Ellis Boyd 'Red' Redding
2  tt0111161      Bob Gunton     Actor             Warden Norton
3  tt0111161  William Sadler     Actor                   Heywood
4  tt0111161    Clancy Brown     Actor            Captain Hadley

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19290 entries, 0 to 19289
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Movie_ID            19290 non-null  object
 1   Name                19290 non-null  object
 2   Role Type           19290 non-null  object
 3   Character/Position  19158 non-null  object
dtypes: object(4)
memory usage: 602.9+ KB
None

Cleaned data saved to clean_full_cast_and_crew.csv


# Transform The Data

## Processing the imbd table

In [13]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler


input_file = "clean_imdb_top_250_movies.csv"
output_file = "processed_imdb_250_movies.csv"

print(f"Loading cleaned data from {input_file}...")


try:
    df = pd.read_csv(input_file)
    print("Data loaded successfully.")
except Exception as e:
    print(f"Error loading file: {e}")
    exit()


print("Data Overview Before Processing:")
print(df.info())
print(df.head())


def encode_categorical(df, column):
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column].astype(str))
    return df


categorical_columns = ['Genre(s)', 'Country of Origin', 'Language', 'Production Companies']


for column in categorical_columns:
    if column in df.columns:
        df = encode_categorical(df, column)


numerical_columns = ['Rating', 'Budget', 'Gross Worldwide Revenue']


for column in numerical_columns:
    if column in df.columns:
        if df[column].isna().all(): 
            print(f"Warning: Column '{column}' contains all NaN values. It will be skipped in normalization.")
            
            df[column] = df[column].fillna(df[column].median())  

scaler = MinMaxScaler()
for column in numerical_columns:
    if column in df.columns and not df[column].isna().all():  # Check if column has valid data
        df[column] = scaler.fit_transform(df[[column]])




columns_to_standardize = ['Budget', 'Gross Worldwide Revenue']


for column in columns_to_standardize:
    if column in df.columns and df[column].isna().all():
        print(f"Warning: Column '{column}' contains all NaN values. It will be skipped in standardization.")
        df[column] = df[column].fillna(df[column].median())  # Fill NaN values with the median

scaler = StandardScaler()
for column in columns_to_standardize:
    if column in df.columns and not df[column].isna().all():  
        df[column] = scaler.fit_transform(df[[column]])


df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].median())


for column in categorical_columns:
    if column in df.columns:
        df[column] = df[column].fillna("Unknown")

try:
    df.to_csv(output_file, index=False)
    print(f"Processed data saved successfully to {output_file}.")
except Exception as e:
    print(f"Error saving processed file: {e}")


print("Processed Data Overview:")
print(df.info())
print(df.head())


Loading cleaned data from clean_imdb_top_250_movies.csv...
Data loaded successfully.
Data Overview Before Processing:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Movie_ID                 250 non-null    object 
 1   Title                    250 non-null    object 
 2   Release Year             250 non-null    int64  
 3   Release Date             250 non-null    object 
 4   Rating                   250 non-null    object 
 5   Director(s)              250 non-null    object 
 6   Genre(s)                 250 non-null    object 
 7   Country of Origin        250 non-null    object 
 8   Language                 249 non-null    object 
 9   Filming Locations        234 non-null    object 
 10  Budget                   229 non-null    float64
 11  Gross Worldwide Revenue  246 non-null    float64
 12  Official Sites  

ValueError: could not convert string to float: '9.3/10'

## Integrate both Tables

In [18]:
import pandas as pd


imdb_df = pd.read_csv("processed_imdb_250_movies.csv")
cast_df = pd.read_csv('full_cast_and_crew.csv')


imdb_df.columns = imdb_df.columns.str.strip()
cast_df.columns = cast_df.columns.str.strip()


imdb_df['Movie_ID'] = imdb_df['Movie_ID'].str.strip().str.upper()
cast_df['Movie_ID'] = cast_df['Movie_ID'].str.strip().str.upper()


imdb_df = imdb_df.drop_duplicates(subset=['Movie_ID'])
cast_df = cast_df.drop_duplicates(subset=['Movie_ID', 'Name', 'Role Type', 'Character/Position'])


cast_df = cast_df[cast_df['Movie_ID'].isin(imdb_df['Movie_ID'])]

# Merge the datasets on 'Movie_ID' to simulate foreign key usage
merged_df = pd.merge(cast_df, imdb_df, on='Movie_ID', how='inner')


print("\nFirst 5 rows of Merged dataset (merged_df):")
print(merged_df.head(5))

merged_df.to_csv('merged_imdb_cast_and_crew.csv', index=False)
print("Merged dataset saved as 'merged_imdb_cast_and_crew.csv'")



First 5 rows of Merged dataset (merged_df):
    Movie_ID            Name Role Type        Character/Position  \
0  TT0111161     Tim Robbins     Actor             Andy Dufresne   
1  TT0111161  Morgan Freeman     Actor  Ellis Boyd 'Red' Redding   
2  TT0111161      Bob Gunton     Actor             Warden Norton   
3  TT0111161  William Sadler     Actor                   Heywood   
4  TT0111161    Clancy Brown     Actor            Captain Hadley   

                      Title  Release Year Release Date  Rating  \
0  The Shawshank Redemption          1994   1994-10-14     NaN   
1  The Shawshank Redemption          1994   1994-10-14     NaN   
2  The Shawshank Redemption          1994   1994-10-14     NaN   
3  The Shawshank Redemption          1994   1994-10-14     NaN   
4  The Shawshank Redemption          1994   1994-10-14     NaN   

      Director(s)  Genre(s)  Country of Origin  Language  \
0  Frank Darabont        75                 46         4   
1  Frank Darabont        75  

## Connection to MySQL

In [15]:
from sqlalchemy import create_engine


username = 'root'            
password = 'Meera#04'  
host = 'localhost'           
port = '3306'               
database = 'mysql'  

engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')


try:
    connection = engine.connect()
    print("Connection to MySQL database established successfully!")
    connection.close()  
except Exception as e:
    print(f"An error occurred: {e}")


Connection to MySQL database established successfully!


In [22]:
from sqlalchemy import create_engine


username = 'root'            
password = 'Meera#04'       
host = 'localhost'           
port = '3306'                


engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}')


new_database = 'movie' 


try:
    with engine.connect() as connection:
        connection.execute(f"CREATE DATABASE IF NOT EXISTS {new_database};")
        print(f"Database '{new_database}' created successfully (or already exists).")
except Exception as e:
    print(f"An error occurred: {e}")


Database 'movie' created successfully (or already exists).


In [23]:
import pandas as pd
from sqlalchemy import create_engine

# Connection details
username = 'root'           
password = 'Meera#04'        
host = 'localhost'           
port = '3306'                
database = 'movie'           


csv_file = 'merged_imdb_cast_and_crew.csv'  
df = pd.read_csv(csv_file)


engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')


try:
    df.to_sql('imdb_data', con=engine, if_exists='replace', index=False)
    print("Data loaded successfully into the 'imdb_data' table.")
except Exception as e:
    print(f"An error occurred: {e}")


Data loaded successfully into the 'imdb_data' table.
